### Dependencias

In [1]:
import cupy as cp
import numpy as np
import pandas as pd

### Función de activación

In [2]:
def leaky_relu(x, alpha=0.01):
    return cp.maximum(alpha * x, x)

def leaky_relu_derivative(x, alpha=0.01):
    dx = cp.ones_like(x)
    dx[x < 0] = alpha
    return dx

def softmax(z):#z: vector resultante antes de funcion de activación en la última capa
    exp_z = cp.exp(z - cp.max(z, axis=1, keepdims=True))  
    return exp_z / cp.sum(exp_z, axis=1, keepdims=True)#retorna las probabilidades de las posibles clases

#mide la diferencia entre distribucion de prob. creada por softmax y los targets reales
def cross_entropy_loss(predictions, targets):
    return -cp.sum(targets * cp.log(predictions + 1e-9)) / targets.shape[0]

### Clase MLP

In [3]:

class MultiLayerNetwork:
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.weights = []
        self.biases = []

        # Initialize weights and biases for each layer
        for i in range(len(layer_sizes) - 1):
            weight = cp.random.randn(layer_sizes[i], layer_sizes[i + 1]) * cp.sqrt(1. / layer_sizes[i])
            bias = cp.zeros((1, layer_sizes[i + 1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, inputs):
        self.activations = [inputs]
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = leaky_relu(z)
            self.activations.append(a)

        # Output uses softmax for multiclass classification
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)
        self.activations.append(a)
        return a

    def predict(self, inputs):
        inputs = cp.array(inputs, ndmin=2)
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = leaky_relu(z)
        
        # Output with softmax
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)

        return a

    def backward(self, targets, learning_rate, clip_value=None):
        delta_weights = [0] * len(self.weights)
        delta_biases = [0] * len(self.biases)
        
        # Calculate the initial error (difference between prediction and target for the output layer)
        error = self.activations[-1] - targets
        
        for i in reversed(range(len(self.weights))):
            # Calculate the delta for the current layer
            delta = error
            delta_weights[i] = cp.dot(self.activations[i].T, delta)
            delta_biases[i] = cp.sum(delta, axis=0, keepdims=True)
            
            if i != 0:
                # Propagate the error to the previous layer
                error = cp.dot(delta, self.weights[i].T) * leaky_relu_derivative(self.activations[i])
            
            # Clip gradients if clip_value is provided
            if clip_value:
                delta_weights[i] = cp.clip(delta_weights[i], -clip_value, clip_value)
                delta_biases[i] = cp.clip(delta_biases[i], -clip_value, clip_value)
            
            # Update weights and biases
            self.weights[i] -= learning_rate * delta_weights[i]
            self.biases[i] -= learning_rate * delta_biases[i]

        #print(self.weights)
        
    def train(self, inputs, targets, epochs, learning_rate, clip_value=None):
        inputs = cp.array(inputs, ndmin=2)
        targets = cp.array(targets, ndmin=2)
        errors = []

        for epoch in range(epochs):
            predictions = self.forward(inputs)
            
            self.backward(targets, learning_rate, clip_value)
            error = cross_entropy_loss(predictions, targets)
            errors.append(error)

            print(predictions)
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Error: {error}')

        return errors

### Data Prep

In [4]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data_2.csv')
train_data = train_data.to_numpy()
    
#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, 1:-1]  # Features for training    
labels = train_data[:train_samples, -1]  #Labels for training

labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

X_train = cp.array(features)
y_train = cp.array(labels, ndmin=2)

print(train_data.shape)
print(features.shape)
print(labels.shape)

(10155, 3074)
(10155, 3072)
(10155, 1)


### Train

In [5]:
num_classes = 10
input_size = 3072
hidden_layers = [256,256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]

model = MultiLayerNetwork(layer_sizes)
epochs = 100
learning_rate = 0.0001

errors = model.train(X_train, y_train, epochs, learning_rate, 2.5)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

[[1.00000000e+000 5.75118358e-023 7.46531849e-052 ... 1.40892032e-019
  2.69626527e-055 6.72151028e-041]
 [1.00000000e+000 2.85011978e-054 9.72615225e-120 ... 3.12227626e-034
  9.95388415e-103 8.76166278e-085]
 [1.00000000e+000 1.12142290e-028 3.70483863e-044 ... 6.39510007e-022
  9.00183458e-055 4.10077246e-065]
 ...
 [1.00000000e+000 2.76471428e-036 3.62234706e-071 ... 2.95317926e-031
  2.22041725e-059 3.75348821e-053]
 [1.00000000e+000 1.83134138e-042 3.38777212e-097 ... 2.11030662e-034
  4.80360398e-084 4.70599808e-056]
 [1.00000000e+000 1.40646306e-030 2.43528675e-102 ... 3.03214506e-028
  2.92160931e-081 2.00487026e-053]]
Epoch 0, Error: 832.6109296796737
[[1.00000000e+000 5.50772862e-020 1.14424230e-064 ... 1.52444924e-042
  4.18841310e-069 8.00993398e-042]
 [1.00000000e+000 9.15861419e-025 5.40552882e-122 ... 7.87737902e-058
  3.32138493e-105 3.61093994e-062]
 [9.90344520e-001 9.65547981e-003 1.29691487e-047 ... 8.55434514e-036
  4.48351886e-055 1.70173543e-039]
 ...
 [1.000000

KeyboardInterrupt: 

## Búsqueda de parámetros

#### Grid Searh para buscar learning rate:


In [11]:

epochs = 100
learning_rate = [10e-15,0.00001, 0.0001,0.001, 0.01, 0.1, 1]

for lr in learning_rate:
    print(f"Learning rate de: {lr}")

    model = MultiLayerNetwork(layer_sizes)
    errors = model.train(X_train, y_train, epochs, lr)

    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f'Test Accuracy: {accuracy}')
    print("")

Learning rate de: 1e-14
Epoch 0, Error: 120.66645697607764
Test Accuracy: 0.07394771870560732

Learning rate de: 1e-05
Epoch 0, Error: 106.33975866854324
Test Accuracy: 0.14046822742474915

Learning rate de: 0.0001
Epoch 0, Error: 107.23506685699824
Test Accuracy: 0.14046822742474915

Learning rate de: 0.001
Epoch 0, Error: 135.5762837276628
Test Accuracy: 0.14046822742474915

Learning rate de: 0.01
Epoch 0, Error: 107.94783862127046
Test Accuracy: 0.14046822742474915

Learning rate de: 0.1
Epoch 0, Error: 121.5481609965781
Test Accuracy: 0.14046822742474915

Learning rate de: 1
Epoch 0, Error: 111.2974164633621
Test Accuracy: 0.14046822742474915



#### Relación entre learning rate y número de épocas

In [12]:

num_classes = 10
input_size = 3072
hidden_layers = [256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]

epochs = [100, 500, 1000, 2500, 5000]
learning_rate = [10e-15, 0.01]
lr_e_results = []
for lr in learning_rate:
    for e in epochs:
        print(f"Learning rate de: {lr}")
        print(f"Epochs: {e}")

        model = MultiLayerNetwork(layer_sizes)
        errors = model.train(X_train, y_train, e, lr)

        #Evaluar
        predictions = model.predict(X_train)
        accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
        print(f'Test Accuracy: {accuracy}')
        print("")
        lr_e_results.append(accuracy)


Learning rate de: 1e-14
Epochs: 100
Test Accuracy: 0.09089383787653382

Learning rate de: 1e-14
Epochs: 500
Test Accuracy: 0.09392512388004609

Learning rate de: 1e-14
Epochs: 1000
Test Accuracy: 0.11373474569635686

Learning rate de: 1e-14
Epochs: 2500
Test Accuracy: 0.1080748537488395

Learning rate de: 1e-14
Epochs: 5000
Test Accuracy: 0.07074864934396707

Learning rate de: 0.01
Epochs: 100
Test Accuracy: 0.14046822742474915

Learning rate de: 0.01
Epochs: 500
Test Accuracy: 0.14046822742474915

Learning rate de: 0.01
Epochs: 1000
Test Accuracy: 0.14046822742474915

Learning rate de: 0.01
Epochs: 2500
Test Accuracy: 0.14046822742474915

Learning rate de: 0.01
Epochs: 5000
Test Accuracy: 0.14046822742474915



#### Grid Searh para ajustar complejitud de 1 capa oculta

In [19]:

epochs = 100
layer_sizes = [input_size] + hidden_layers + [output_size]
lr = 0.00001
for i in range(0, 5):  # Cambia el 5 por el número de potencias de 10 que desees
    val = pow(10, i)
    hidden_layers = [val]  # Tamaños de las capas ocultas
    print(f"layer size: {val}")
    layer_sizes = [input_size] + hidden_layers + [output_size]
    print(layer_sizes)

    model = MultiLayerNetwork(layer_sizes)
    errors = model.train(X_train, y_train, epochs, lr)

    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f'Test Accuracy: {accuracy}')
    print("")

layer size: 1
[3072, 1, 10]
Epoch 0, Error: 102.89002775587444
Test Accuracy: 0.11705685618729098

layer size: 10
[3072, 10, 10]
Epoch 0, Error: 107.33877121476301
Test Accuracy: 0.14046822742474915

layer size: 100
[3072, 100, 10]
Epoch 0, Error: 105.03349595272618
Test Accuracy: 0.0903010033444816

layer size: 1000
[3072, 1000, 10]
Epoch 0, Error: 112.76559406381097
Test Accuracy: 0.14046822742474915

layer size: 10000
[3072, 10000, 10]
Epoch 0, Error: 116.52473388148074
Test Accuracy: 0.0903010033444816



### Grid Search para ajustar numero de capas ocultas

In [18]:

num_classes = 10
input_size = 3072
hidden_layer_size = 256
hidden_layers = []  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 100
lr = 0.0001

for i in range(0, 10):  # Cambia el 5 por el número de potencias de 10 que desees
    hidden_layers.append(hidden_layer_size)  # Tamaños de las capas ocultas
    print(f"hidden layers: {hidden_layers}")
    layer_sizes = [input_size] + hidden_layers + [output_size]

    model = MultiLayerNetwork(layer_sizes)
    errors = model.train(X_train, y_train, epochs, lr)

    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f'Test Accuracy: {accuracy}')
    print("")

hidden layers: [256]
Epoch 0, Error: 105.86025886841283
Test Accuracy: 0.09698996655518395

hidden layers: [256, 256]
Epoch 0, Error: 108.85456644116445
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256]
Epoch 0, Error: 105.7057006341719
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256]
Epoch 0, Error: 110.30727016766471
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256]
Epoch 0, Error: 125.7849884262953
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256]
Epoch 0, Error: 111.10845764177878
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 106.9112295635407
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 107.25215446902249
Test Accuracy: 0.14046822742474915

hidden layers: [256, 256, 256, 256, 256, 256, 256, 256, 256]
Epoch 0, Error: 105.26643352242563
Test Accuracy: 0.14046822742474

### Modelo entrenado con parámetros encontrados

In [15]:
num_classes = 10
input_size = 3072
hidden_layers = [10, 10]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 500
learning_rate = 0.0001

model = MultiLayerNetwork(layer_sizes)
errors = model.train(X_train, y_train, epochs, learning_rate)
print(errors)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')


[array(107.19914646), array(691.01418078), array(810.28662508), array(810.28662508), array(810.28662508), array(252.3364584), array(726.27750819), array(252.3364584), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array(nan), array

## Regularizaciones:

#### Dropout


In [25]:

class MultiLayerNetwork_Dropout:
    def __init__(self, layer_sizes, dropout_rate):
        self.layer_sizes = layer_sizes
        self.weights = []
        self.biases = []
        self.dropout_rate = dropout_rate

        # Initialize weights and biases for each layer
        for i in range(len(layer_sizes) - 1):
            weight = cp.random.randn(layer_sizes[i], layer_sizes[i + 1]) * cp.sqrt(2. / layer_sizes[i])
            bias = cp.zeros((1, layer_sizes[i + 1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, inputs, training=True):
        self.activations = [inputs]
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = relu(z)
            # Apply dropout during training
            if training:
                dropout_mask = cp.random.binomial(1, 1 - self.dropout_rate, size=a.shape) / (1 - self.dropout_rate)
                a *= dropout_mask
            self.activations.append(a)
        
        # Output with softmax
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)
        self.activations.append(a)

        return a

    def predict(self, inputs):
        inputs = cp.array(inputs, ndmin=2)
        a = inputs

        for i in range(len(self.weights) - 1):
            z = cp.dot(a, self.weights[i]) + self.biases[i]
            a = relu(z)
        
        # Output with softmax
        z = cp.dot(a, self.weights[-1]) + self.biases[-1]
        a = softmax(z)

        return a

    def backward(self, targets, learning_rate):
        delta_weights = [0] * len(self.weights)
        delta_biases = [0] * len(self.biases)
        
        # Calculate the initial error (difference between prediction and target for the output layer)
        error = self.activations[-1] - targets
        
        for i in reversed(range(len(self.weights))):
            # Calculate the delta for the current layer
            delta = error
            delta_weights[i] = cp.dot(self.activations[i].T, delta)
            delta_biases[i] = cp.sum(delta, axis=0, keepdims=True)
            
            if i != 0:
                # Propagate the error to the previous layer
                error = cp.dot(delta, self.weights[i].T) * relu_derivative(self.activations[i])
            
            # Update weights and biases
            self.weights[i] -= learning_rate * delta_weights[i]
            self.biases[i] -= learning_rate * delta_biases[i]


    def train(self, inputs, targets, epochs, learning_rate):
        inputs = cp.array(inputs, ndmin=2)
        targets = cp.array(targets, ndmin=2)
        errors = []

        for epoch in range(epochs):
            predictions = self.forward(inputs, training=True)
            self.backward(targets, learning_rate)
            error = cross_entropy_loss(predictions, targets)
            errors.append(error)

            #if epoch % 100 == 0:
            #    print(f'Epoch {epoch}, Error: {error}')

        return errors

In [13]:
num_classes = 10
input_size = 3072
hidden_layers = [256,256,256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = [1, 5, 10, 100, 500, 1000]
learning_rate = 0.00001
dropout_rate = 0.2

for e in epochs:
    model = MultiLayerNetwork_Dropout(layer_sizes, dropout_rate)
    errors = model.train(X_train, y_train, e, learning_rate)
    #Evaluar
    predictions = model.predict(X_train)
    accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
    print(f"epochs: {e}")
    print(f'Test Accuracy: {accuracy}')
    print("")

CUDARuntimeError: cudaErrorLaunchTimeout: the launch timed out and was terminated

# No mejora nada k chucha ( )

### Feature engineering

#### Porcentaje para cada canal RGB

In [11]:
red_pixels = []
green_pixels = []
blue_pixels = []
train_data = train_data[:, 1:-1]

# Iterate through each row in the original matrix
for row in train_data:
    pr = []
    pg = []
    pb = []
    for i in range(len(row)):
        if i % 3 == 0:
            pr.append(row[i])
        
        if i % 3 == 1:
            pg.append(row[i])

        if i % 3 == 2:
            pb.append(row[i])

    red_pixels.append(pr)
    green_pixels.append(pg)
    blue_pixels.append(pb)


#### Más datos

In [12]:
#intensidad media por conal
mean_red = np.mean(red_pixels, axis=1)
mean_green = np.mean(green_pixels, axis=1)
mean_blue = np.mean(blue_pixels, axis=1)

#varianza
var_red = np.var(red_pixels, axis=1)
var_green = np.var(green_pixels, axis=1)
var_blue = np.var(blue_pixels, axis=1)

#desviación estándar
std_red = np.std(red_pixels, axis=1)
std_green = np.std(green_pixels, axis=1)
std_blue = np.std(blue_pixels, axis=1)

#contraste
contrast_red = np.ptp(red_pixels, axis=1) # ptp: peak to peak (max - min)
contrast_green = np.ptp(green_pixels, axis=1)
contrast_blue = np.ptp(blue_pixels, axis=1)

channel_data = np.column_stack((
                                mean_red, mean_green, mean_blue,
                                var_red, var_green, var_blue,
                                std_red, std_green, std_blue,
                                contrast_red, contrast_green, contrast_blue
                                ))

print(channel_data.shape)



(10155, 12)


### train

In [14]:
#Cortar en features y labales
channel_data_train_samples = channel_data.shape[0]
features = channel_data  # Features for training
labels = train_data[:train_samples, -1]   # Labels for training

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

print(channel_data.shape)
print(features.shape )
print(labels.shape)


indices = cp.arange(channel_data_train_samples)
X_train = cp.array(features)
y_train = cp.array(labels)



num_classes = 10
input_size = 12
hidden_layers = [256,256]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]
epochs = 100
learning_rate = 0.01
dropout_rate = 0.2

model = MultiLayerNetwork(layer_sizes)
errors = model.train(X_train, y_train, epochs, learning_rate, 2.5)
#Evaluar
predictions = model.predict(X_train)

accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

(10155, 12)
(10155, 12)
(10155, 1)
[[1.00000000e+000 1.33933645e-278 1.24641384e-068 ... 2.06991234e-225
  6.92848298e-245 0.00000000e+000]
 [1.00000000e+000 0.00000000e+000 7.94968473e-289 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000 0.00000000e+000 1.26475152e-178 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [1.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000 4.02727413e-315 4.37785181e-087 ... 1.14765653e-278
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000 0.00000000e+000 9.10669404e-284 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]]
Epoch 0, Error: 23565.77791297299
[[7.21052142e-228 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+0

#### Early stopping

## UTIL


### Data prep pero cortando 20% para test

In [ ]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data_2.csv')
train_data = train_data.to_numpy()

#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, :-1]  # Features for training
labels = train_data[:train_samples, -1]   # Labels for training

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

print(train_data.shape)
print(features.shape )
print(labels.shape)


indices = cp.arange(train_samples)

#cortar 20% test 
test_size = 0.2
test_samples = int(test_size * train_samples)
train_indices, test_indices = indices[test_samples:], indices[:test_samples]

X_train, X_test = features[train_indices], features[test_indices]
y_train, y_test = labels[train_indices], labels[test_indices]

train_indices = cp.array(train_indices)
test_indices = cp.array(test_indices)
X_train = cp.array(X_train)
X_test = cp.array(X_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [7]:
#Obtener datos (estoy usando pd porque anda considerablemente más rápido que np)
train_data = pd.read_csv(r'C:/Users/kueru/Documents/VSCode/semestre_9/Deep_Learning/T2/train_data.csv')
train_data = train_data.to_numpy()
    
#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, 1:-1]  # Features for training
labels = train_data[:train_samples, -1]   # Labels for training

# Reduce pixel values
features = features / 255.0 
# flatten the label values
labels = labels.flatten()

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

print(train_data.shape)
print(features.shape )
print(labels.shape)


indices = cp.arange(train_samples)


X_train = cp.array(features)
y_train = cp.array(labels)

num_classes = 10
input_size = 3072
hidden_layers = [100, 100]  # Tamaños de las capas ocultas
output_size = 10
layer_sizes = [input_size] + hidden_layers + [output_size]

model = MultiLayerNetwork(layer_sizes)
epochs = 1
learning_rate = 10e-15

errors = model.train(X_train, y_train, epochs, learning_rate)
print(errors)
#Evaluar
predictions = model.predict(X_train)
accuracy = cp.mean(predictions.argmax(axis=1) == y_train)
print(f'Test Accuracy: {accuracy}')

KeyboardInterrupt: 